In [1]:
%run src_runs/widget.py


Initiating project interface...
Project interface initiated successfully.


Script cmd : start cmd /k python e:\workspace\learndl\src_runs\0_git\1_pull_only.py --email 0
Script cmd : start cmd /k python e:\workspace\learndl\src_runs\1_autorun\0_daily_update.py --email 1
